In [1]:
from pymongo import MongoClient

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','companies')

In [4]:
companies = list(coll.find({"offices.city": "San Jose","founded_year": {"$lt": 2008},"offices.latitude":{"$ne": None},"offices.longitude":{"$ne":None}}))

In [5]:
print(len(companies))

89


In [6]:
print(companies[0]['offices'][0]['latitude'],companies[0]['offices'][0]['longitude'])

37.295005 -121.930035


In [7]:
set(companies[0]['offices'][0])

{'address1',
 'address2',
 'city',
 'country_code',
 'description',
 'latitude',
 'longitude',
 'state_code',
 'zip_code'}

In [8]:
def getLocation(company):
    location=[]
    for i in range(len(company['offices'])):
        latitude = company['offices'][i]['latitude']
        longitude = company['offices'][i]['longitude']
        loc = {
            'type':'Point',
            'coordinates':[longitude,latitude]
        }
        location.append(loc)
    return location

In [9]:
for company in companies:
    value = {"$set": {'location':getLocation(company)}}
    coll.update_one(company,value)

In [10]:
companies[0]['location']

[{'type': 'Point', 'coordinates': [-121.930035, 37.295005]}]

In [11]:
companies[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
 'name': 'eBay',
 'permalink': 'ebay',
 'crunchbase_url': 'http://www.crunchbase.com/company/ebay',
 'homepage_url': 'http://ebay.com',
 'blog_url': '',
 'blog_feed_url': '',
 'twitter_username': 'eBay',
 'category_code': 'web',
 'number_of_employees': 15000,
 'founded_year': 1995,
 'founded_month': 9,
 'founded_day': 1,
 'deadpooled_year': None,
 'deadpooled_month': None,
 'deadpooled_day': None,
 'deadpooled_url': None,
 'tag_list': 'auction, bid, search, market',
 'alias_list': '',
 'email_address': '',
 'phone_number': '',
 'description': 'Online Marketplace',
 'created_at': 'Wed Jun 06 13:03:13 UTC 2007',
 'updated_at': 'Tue Sep 10 03:38:03 UTC 2013',
 'overview': '<p>Founded in 1995 in San Jose, CA, eBay connects millions of buyers and sellers globally in the world&#8217;s largest online marketplace, utilizing <a href="/company/paypal" title="PayPal" rel="nofollow">PayPal</a> to ensure secure transactions. The company also operates sp

In [12]:
import folium

In [13]:
map_city=folium.Map(location=[37.3382,-121.8863], zoom_start=12)
for company in companies:
    folium.Marker(company['location'][0]['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)

In [14]:
map_city